# Running Back Analysis

In [1]:
import duckdb
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
DATABASE = "../database.db"
con = duckdb.connect(DATABASE, read_only=True)

In [16]:
query = """
SELECT
    season,
    week,
    category,
    player_id,
    player_name,
    position,
    team,
    opponent,
    ppr_scoring_fantasy_points,
    rushing_attempts,
    rushing_yards,
    rushing_touchdowns,
    receiving_targets,
    receiving_yards,
    receiving_touchdowns
FROM player_week_data
WHERE
    category = 'stat'
    AND season = 2023
    AND position = 'RB'
LIMIT 10
"""
con.execute(query).fetchdf()

,season,week,category,player_id,player_name,position,team,opponent,ppr_scoring_fantasy_points,rushing_attempts,rushing_yards,rushing_touchdowns,receiving_targets,receiving_yards,receiving_touchdowns
0,2023,1,stat,10216,Kenny McIntosh,RB,SEA,LAR,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2023,1,stat,10219,Chris Rodriguez,RB,WAS,ARI,0.7,3.0,7.0,0.0,0.0,0.0,0.0
2,2023,1,stat,10223,Eric Gray,RB,NYG,DAL,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2023,1,stat,10235,Roschon Johnson,RB,CHI,GB,17.5,5.0,20.0,1.0,7.0,35.0,0.0
4,2023,1,stat,1034,Brandon Bolden,RB,LV,DEN,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,2023,1,stat,10861,Xazavian Valladay,RB,NYJ,BUF,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,2023,1,stat,10870,Deneric Prince,RB,KC,DET,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,2023,1,stat,11060,Robert Burns,RB,CHI,GB,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,2023,1,stat,11104,SaRodorick Thompson,RB,SEA,LAR,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,2023,1,stat,11114,Elijah Dotson,RB,LAC,MIA,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [54]:
query = """
WITH rusher_week_data AS (
    SELECT
        *,
        IF(rushing_yards < 0, 0, rushing_yards) AS non_zero_rushing_yards,
        IF(rushing_attempts > 0, rushing_yards / rushing_attempts, null) AS rushing_yards_per_attempt
    FROM player_week_data
    WHERE
        category = 'stat'
        AND season = 2023
        AND position = 'RB'
        AND rushing_attempts > 0
),

team_week_rushing_data AS (
    SELECT
        season,
        week,
        team,
        SUM(rushing_attempts) AS team_rushing_attempts,
        SUM(non_zero_rushing_yards) AS team_rushing_yards,
        ARGMAX(player_id, rushing_attempts) AS top_rusher_by_attempts,
        ARGMAX(player_id, rushing_yards) AS top_rusher_by_yards,
        MAX(rushing_yards_per_attempt) AS top_rushing_yards_per_attempt,
        MAX(rushing_attempts) AS top_rush_attempts,
        MAX(non_zero_rushing_yards) AS top_rush_yards
    FROM rusher_week_data
    GROUP BY
        season,
        week,
        team
),

rusher_week_share_data AS (
    SELECT
        r.*,
        r.rushing_attempts / t.team_rushing_attempts AS fraction_of_rush_attempts,
        r.non_zero_rushing_yards / t.team_rushing_yards AS fraction_of_rush_yards,
        r.player_id = t.top_rusher_by_attempts AS is_top_rusher_by_attempts,
        r.player_id = t.top_rusher_by_yards AS is_top_rusher_by_yards,
        t.* EXCLUDE (season, week, team)
    FROM rusher_week_data r
    LEFT JOIN team_week_rushing_data t
        ON r.season = t.season
        AND r.week = t.week
        AND r.team = t.team
    ORDER BY
        r.season,
        r.week,
        r.team,
        fraction_of_rush_attempts DESC
)

SELECT
    week,
    team,
    opponent,
    player_id,
    player_name,
    ppr_scoring_fantasy_points,
    fraction_of_rush_attempts,
    fraction_of_rush_yards,
    rushing_attempts,
    rushing_yards
FROM rusher_week_share_data
WHERE
    top_rusher_by_attempts != top_rusher_by_yards
    AND (player_id = top_rusher_by_attempts OR player_id = top_rusher_by_yards)
LIMIT 50
"""
con.execute(query).fetchdf()

,week,team,opponent,player_id,player_name,ppr_scoring_fantasy_points,fraction_of_rush_attempts,fraction_of_rush_yards,rushing_attempts,rushing_yards
0,1,GB,CHI,6828,AJ Dillon,5.6,0.481481,0.231707,13.0,19.0
1,1,GB,CHI,4199,Aaron Jones,26.7,0.333333,0.500000,9.0,41.0
2,1,LAR,SEA,6938,Cam Akers,8.9,0.594595,0.358025,22.0,29.0
3,1,LAR,SEA,8150,Kyren Williams,17.4,0.405405,0.641975,15.0,52.0
4,1,NE,PHI,7611,Rhamondre Stevenson,14.9,0.600000,0.409836,12.0,25.0
5,1,NE,PHI,3164,Ezekiel Elliott,7.3,0.350000,0.475410,7.0,29.0
6,1,NYJ,BUF,4029,Dalvin Cook,8.9,0.541667,0.198795,13.0,33.0
7,1,NYJ,BUF,8155,Breece Hall,15.7,0.416667,0.765060,10.0,127.0
8,2,BAL,CIN,5995,Justice Hill,8.3,0.523810,0.398058,11.0,41.0
9,2,BAL,CIN,5248,Gus Edwards,12.2,0.476190,0.601942,10.0,62.0
